In [1]:
import numpy as np
import onnx
import os
import glob
from onnx import numpy_helper
import tvm
from tvm import te
import tvm.relay as relay

In [2]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [3]:
# load module param and mod

In [4]:
input_name = "Input3"
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [5]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [6]:
# target = tvm.target.arm_cpu("rasp3b")
# target = tvm.target.create('llvm')
#  target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
target = tvm.target.intel_graphics()
# target = tvm.target.Target("opencl -device=intel_graphics")
# target = tvm.target.Target("opencl -device=intel_graphics")

In [7]:
print(target)

opencl -keys=intel_graphics,opencl,gpu -device=intel_graphics -max_num_threads=256 -model=unknown -thread_warp_size=16


In [8]:
target_host = "llvm"

In [9]:
with tvm.transform.PassContext(opt_level=3):
# with relay.build_config(opt_level=3):    
    module_lib = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mnist')

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [10]:
print(module_lib)

In [11]:
# lib export
lib_path = "../lib/mnist.so"

In [12]:
module_lib.export_library(lib_path)
#cc="/usr/bin/aarch64-linux-gnu-g++"
# cc="/usr/bin/arm-linux-gnueabihf-g++

In [13]:
# params export

In [14]:
param_path = "../lib/mnist.params"

In [15]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [16]:
ctx = tvm.opencl()

In [18]:
module = tvm.contrib.graph_executor.GraphModule(module_lib["mnist"](ctx))

TVMError: Traceback (most recent call last):
  8: TVMFuncCall
  7: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::GraphExecutorFactory::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  6: tvm::runtime::GraphExecutorFactory::ExecutorCreate(std::vector<DLDevice, std::allocator<DLDevice> > const&)
  5: tvm::runtime::GraphExecutor::Init(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::Module, std::vector<DLDevice, std::allocator<DLDevice> > const&, tvm::runtime::PackedFunc)
  4: tvm::runtime::GraphExecutor::SetupStorage()
  3: tvm::runtime::NDArray::Empty(std::vector<long, std::allocator<long> >, DLDataType, DLDevice, tvm::runtime::Opt